# Bayesian Poisson Model for 33Na Decay Histogram
This notebook implements a Poisson-based Bayesian model to estimate the parent half-life and neutron emission branching ratios for the decay of 33Na.

In [ ]:
# Step 1: Load libraries
import pandas as pd
import pymc as pm
import aesara.tensor as at
import numpy as np
import matplotlib.pyplot as plt
import arviz as az
import seaborn as sns

In [ ]:
# Step 2: Load and clean data
df = pd.read_csv('hist_33Na_with_error.csv', sep='	', header=None, names=['x', 'y', 'yerr'])
df = df.iloc[1:].copy()
df['x'] = pd.to_numeric(df['x'], errors='coerce')
df['y'] = pd.to_numeric(df['y'].str.replace(',', ''), errors='coerce')
df['yerr'] = pd.to_numeric(df['yerr'].str.replace(',', ''), errors='coerce')
df.dropna(inplace=True)

# Define background and decay regions
background_y = df[df['x'] < 0]['y']
bkg_est = np.mean(background_y)
decay_df = df[df['x'] >= 0]
x = decay_df['x'].values
y = decay_df['y'].values

In [ ]:
# Step 3: Define model using Poisson likelihood
with pm.Model() as model:
    
    # Priors
    initialActivity = pm.Gamma('initialActivity', alpha=100, beta=0.003, initval=36000, transform=None)
    background = pm.Gamma('background', alpha=25, beta=0.003, initval=bkg_est, transform=None)
    
    parentT = pm.Gamma('parentT', alpha=64, beta=8, initval=8.0, transform=None)
    daughT = [6.4, 7.2, 11.4]  # ms, fixed
    
    theta = pm.Normal('theta', mu=0, sigma=2, shape=3)
    nBranch = pm.Deterministic('nBranch', at.nnet.softmax(theta))
    
    # Build expected counts
    lam_parent = at.log(2) / parentT
    lam_daugh = [np.log(2) / t for t in daughT]
    
    mu = (initialActivity * at.exp(-lam_parent * x) +
          initialActivity * nBranch[0] * at.exp(-lam_daugh[0] * x) +
          initialActivity * nBranch[1] * at.exp(-lam_daugh[1] * x) +
          initialActivity * nBranch[2] * at.exp(-lam_daugh[2] * x) +
          background)
    
    y_obs = pm.Poisson('y_obs', mu=mu, observed=y)

    # Sample
    trace = pm.sample(1000, tune=1000, chains=2, target_accept=0.95)

In [ ]:
# Step 4: Posterior analysis
az.plot_trace(trace)
az.plot_posterior(trace, var_names=['parentT', 'initialActivity', 'background', 'nBranch'])
plt.show()